In [1]:
path_to_rtss = r"C:\Users\r.joshi\Downloads\01_11_2024\7139000004\COMBI\L"
output_path =r"./"

Modality Configuration Skeleton Generation

In [3]:
from typing import (Tuple, Optional)

import pyradise.data as ps_data
import pyradise.fileio as ps_io 
import pyradise.process as ps_proc

In [4]:
def get_pipeline(output_size: Tuple[int, int, int] = (256, 256, 256),
                 output_spacing: Tuple[float, float, float] = (1.0, 1.0, 1.0),
                 reference_modality: str = 'T1'
                 ) -> ps_proc.FilterPipeline:
    # Create an empty filter pipeline
    pipeline = ps_proc.FilterPipeline()

    # Add an orientation filter to the pipeline
    orientation_params = ps_proc.OrientationFilterParams(output_orientation='RAS')
    pipeline.add_filter(ps_proc.OrientationFilter(), orientation_params)

    # Add a resampling filter to the pipeline
    resample_params = ps_proc.ResampleFilterParams(output_size,
                                                   output_spacing,
                                                   reference_modality=reference_modality,
                                                   centering_method='reference')
    pipeline.add_filter(ps_proc.ResampleFilter(), resample_params)

    return pipeline

In [6]:
ps_io.DatasetDicomCrawler(path_to_rtss, write_modality_config=True).execute()

The modality configuration file already exists and will not be overwritten.


((<pyradise.fileio.series_info.DicomSeriesImageInfo at 0x20445ee25d0>,
  <pyradise.fileio.series_info.DicomSeriesRTSSInfo at 0x2046b4a2990>),)

In [ ]:
def convert_dicom_to_nifti_with_modality_config(input_path: str,
                                                output_path: str
                                                ) -> None:
    # Instantiate a new loader
    loader = ps_io.SubjectLoader()

    # (optional)
    # Get the filter pipeline
    pipeline = get_pipeline()

    # Instantiate a new writer with default settings
    # Note: You can adjust here the output image file format
    # and the naming of the output files
    writer = ps_io.SubjectWriter()

    # (optional)
    # Instantiate a new selection to exclude additional SeriesInfo entries
    expected_modalities = ('T1', 'T2')
    modality_selection = ps_io.ModalityInfoSelector(expected_modalities)

    # Search DICOM files for each subject and iterate over the crawler
    crawler = ps_io.DatasetDicomCrawler(input_path)
    for series_info in crawler:
        # (optional)
        # Keep just the selected modalities for loading
        # Note: SeriesInfo entries for non-image data get returned unfiltered
        # series_info = modality_selection.execute(series_info)
        

        # Load the subject from the series info
        subject = loader.load(series_info)

        reg_info = ps_io.DicomSeriesRegistrationInfo(r"C:\Users\r.joshi\Downloads\01_11_2024\7139000003\COMBI\L\ultrasound.0.dcm", subject)

        # (optional)
        # Execute the filter pipeline on the subject
        print(f'Processing subject {subject.get_name()}...')
        # subject = pipeline.execute(subject)

        # Write each subject to a specific subject directory
        writer.write_to_subject_folder(output_path, subject,
                                       write_transforms=False)

In [28]:
# Execute the conversion procedure (approach 1)
convert_dicom_to_nifti_with_modality_config(path_to_rtss,
                                            output_path)

AttributeError: 'DicomSeriesRTSSInfo' object has no attribute 'get'

In [15]:
rtss_info = ps_io.DicomSeriesRTSSInfo(r"C:\Users\r.joshi\Downloads\01_11_2024\7139000003\COMBI\L\rtss.dcm")

In [22]:
rtss_info.get_annotator().get_name()

'NA'

TypeError: DicomSeriesRegistrationInfo.__init__() missing 1 required positional argument: 'image_infos'